In [ ]:
import numpy as np
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D
import keras.optimizers
from keras.utils import np_utils
import matplotlib.pyplot as plt
import cv2
from keras import backend as K

K.set_image_dim_ordering('th')

seed = 7
np.random.seed(seed)
img_size=28
img_shape=(img_size,img_size)

# load data

(X_train, y_train), (X_test, y_test) = mnist.load_data()
x=X_test[3]
# reshape to be [samples][pixels][width][height]
X_train = X_train.reshape(X_train.shape[0],1, 28, 28).astype('float32')
X_test = X_test.reshape(X_test.shape[0], 1, 28, 28).astype('float32')
X_train = X_train / 255
X_test = X_test / 255

# one hot encode
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

def input_img(origimg):
    img = cv2.imread(origimg)
    cv2.imshow('Initial image', img)
    cv2.waitKey(0)
    # cv2.destroyAllWindows()
    return img

def resize_img(img):
    resize_img=cv2.resize(img,(28,28),interpolation = cv2.INTER_CUBIC)
    gray_img=cv2.cvtColor(resize_img,cv2.COLOR_BGR2GRAY)
    #cv2.waitKey(0)
    return gray_img

def tranformTo_binary(img):
    img=cv2.GaussianBlur(img,(3,3),0)
    guassianthre = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,2)
    titles = ['Original Image','Adaptive Gaussian Thresholding']
    images = [img,guassianthre]
    for i in xrange(2):
        plt.subplot(2, 2, i + 1), plt.imshow(images[i], 'gray')
        plt.title(titles[i])
        plt.xticks([]), plt.yticks([])
    plt.show()
    return guassianthre

def pro_img(img):
    img=resize_img(img)
    img = tranformTo_binary(img)


def plot_image(curve,epoch):
    plt.style.use("ggplot")
    plt.figure()
    plt.plot(np.arange(0, epoch), curve.history["loss"], label="train_loss")
    plt.plot(np.arange(0, epoch), curve.history["val_loss"], label="val_loss")
    plt.plot(np.arange(0, epoch), curve.history["acc"], label="train_acc")
    plt.plot(np.arange(0, epoch), curve.history["val_acc"], label="val_acc")
    plt.title("Training Loss and Accuracy")
    plt.xlabel("Epoch #")
    plt.ylabel("Loss/Accuracy")
    plt.legend(loc="upper right")
    plt.show()



def baseline_model():
    # create model
    model = Sequential()
    model.add(Convolution2D(32, 5, 5, border_mode='valid', input_shape=(1, 28, 28), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Convolution2D(64, 5, 5, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.4))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))

    opt =keras.optimizers.rmsprop(lr=0.001)
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    curve = model.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=5, batch_size=200, verbose=2)

    plot_image(curve, 5)
    #
    scores = model.evaluate(X_test, y_test, verbose=0)
    model.save('ML')
    print("Baseline Error: %.2f%%" % (100 - scores[1] * 100))
    print '\nclass:\t',h
    plt.show(x)
    plt.show()
    return model

def main():
    model = baseline_model()
# model = baseline_model()
# curve=model.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=5, batch_size=200, verbose=2)
#
# plot_image(curve,5)
# scores = model.evaluate(X_test, y_test, verbose=0)
# print("Baseline Error: %.2f%%" % (100 - scores[1] * 100))
if __name__ == "__main__":
    main()